In [1]:
using StateSpaceDynamics

In [2]:
using StateSpaceDynamics: initialize_slds
using StateSpaceDynamics: sample

In [3]:
true_ssm = initialize_slds()

SwitchingLinearDynamicalSystem([0.5686413682169027 0.4313586317830972; 0.39042143546215796 0.6095785645378421], StateSpaceDynamics.LinearDynamicalSystem[StateSpaceDynamics.LinearDynamicalSystem{StateSpaceDynamics.GaussianStateModel{Float64}, StateSpaceDynamics.GaussianObservationModel{Float64}}(StateSpaceDynamics.GaussianStateModel{Float64}([0.9204668006251124 -0.2350337612917968; 0.2350337612917968 0.9204668006251124], [0.1 0.0; 0.0 0.1], [0.0, 0.0], [0.1 0.0; 0.0 0.1]), StateSpaceDynamics.GaussianObservationModel{Float64}([-0.8595553820616212 -1.6658075498442124; -1.4692882055065464 0.38163694535441856; … ; 0.517130881450359 1.1979596698125403; 1.4472238915818758 -1.2290887130815225], [0.5 0.0 … 0.0 0.0; 0.0 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.5 0.0; 0.0 0.0 … 0.0 0.5]), 2, 10, Bool[1, 1, 1, 1, 1, 1]), StateSpaceDynamics.LinearDynamicalSystem{StateSpaceDynamics.GaussianStateModel{Float64}, StateSpaceDynamics.GaussianObservationModel{Float64}}(StateSpaceDynamics.GaussianStateModel{Float64}

In [4]:
T = 100
# Generate synthetic data
_, observations, _ = sample(true_ssm, T)


([-0.011827244398846836 0.2788244678457647 … -0.5135485402627521 -0.7222594414897752; -0.22639925721218238 -0.6176799382452005 … -0.02911931950838531 -0.5959456259990294], [1.0441932962440357 1.0861809232869089 … -1.300954969059553 1.96928608983541; -0.8758466925784547 -0.26292180897649947 … 0.7554874898897548 1.257722417240287; … ; -1.3624159771569548 -1.3162247190750846 … 0.16772508831952265 -0.9174156153303139; -1.428861222928581 1.5586739277867219 … -0.3748529538780018 0.09887630441079143], [2, 2, 1, 1, 1, 2, 2, 2, 1, 2  …  2, 2, 2, 2, 1, 2, 2, 2, 2, 2])

In [5]:
using StateSpaceDynamics: variational_expectation

In [6]:
variational_expectation(true_ssm, observations)

([0.8023279156643639 0.5334122626391126 … 0.47509236476244066 0.47509236476244066; 0.1976720843355993 0.4665877373609078 … 0.5249076352376105 0.5249076352376105], [0.0475711321096286 0.00012569094083022828 … 0.0 0.0; 0.01172025135200156 0.0001099447009309059 … 0.0 0.0], [0.0 0.0 … 6.31109390544976e-5 1.0; 0.0 0.0 … 6.31109390544976e-5 1.0], [0.0 0.023470562043173 … -0.1554613767447383 -0.6344428090760141; 0.0 -0.716040365559033 … 0.48841406528176157 -0.43557054322960187], [[0.0 0.0; 0.0 0.0], [0.036732071707220065 -0.00168934352555068; -0.0016893435255506853 0.04494057048068317], [0.03400855997029205 -0.0016736227122747544; -0.001673622712274735 0.041383282177555034], [0.033844781568166775 -0.0016491464731779064; -0.0016491464731778873 0.04109989501888038], [0.03383374247503284 -0.00164491217396432; -0.0016449121739643105 0.041078002859776606], [0.03383290453303239 -0.0016444160395558166; -0.0016444160395558283 0.04107640209722005], [0.033832835733116205 -0.0016443688051182455; -0.0016

In [ ]:
fit_slds!(slds, observations)

In [ ]:
using Plots